# First Order Motion(FOM)

When I started learn about computer since I was a kid, other than computer networking and programming I also interested with multimedia. Since I don't have the skills and talent in art I don't know how to use those fancy tools. Nowadays you don't need to have skills and talent in art you can just ask your program to do what you want based on example provide to your program. For this blog post, I will try generate an animation based on example I will give to the program.

## What is First Order Motion

Given one image of a person, we can manipulate that image to animate according to driving animation(moving image)

Training procedure for FOM is simply pretrain by disentangling the appearance and motion
using source image and driving video.

1. "Motion extraction" model will generate 'dense motion' and 'occlusion' to be pass to the generated image.
2. "Image Generator" will generate an image and that image will be warp by dense motion and then occlusion map will be use for inpainting when the character move around to fix the background.



Inside "Motion Extraction" model there is keypoint extraction model to extract keypoint like face, eyes, nose, mouth, facial contour, etc(This is keypoint model during 2019, if we replace with current one I think the model will perform better in term of tracking). Then from the keypoint it will be use as information for generating the right pixel.

When you character around there will be missing information at the background, that's why with occlusion map it will fix the background to make sure the animation look natural.

Thanks to the dense motion and occlusion we can use any image for inference as long it follow same viewpoint like person facing to the right or anything similar in the dataset.

![](./artifacts/a.png)

During inference you can use any source image and driving animation and it will be able to generate target animation which the motion coming from driving animation and the person is from source image

![](./artifacts/inference.png)

## Code

In [1]:
%cd ../first-order-model

/Users/osmojave/work/blog/first-order-model


In [2]:
import imageio
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [3]:
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
import imageio
from skimage.transform import resize

source_image = imageio.imread('../_notebooks/artifacts/ben_afflek/httpimagesfandangocomrImageRendererredesignstaticimgnoxportraitjpgpcpcpcimagesmasterrepositoryperformerimagespjpg.jpg')
driving_video = imageio.mimread('./sample/04.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='./sample/vox-cpk.pth.tar',cpu=True)

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,cpu=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 211/211 [06:02<00:00,  1.72s/it]


In [4]:
HTML(display(source_image, driving_video, predictions).to_html5_video())

As you can see there is no audio since the model can only generate sequence of image. You can extract original video audio and merge it with the video if you want to make the video with audio.

## Final thoughts

First order motion is a model from 2019 which is quite old if you in AI industry. I think we can do lot more cool stuff with current state of the art model that quite accurate and small.

Reference:
1. https://aliaksandrsiarohin.github.io/first-order-model-website/
2. https://github.com/cedro3/first-order-model
3. https://github.com/AliaksandrSiarohin/first-order-model